# Exploratory Data Analysis
This noteboook serves as initial analysis on the dataset to inspect properties of data and rationalize any necessary treatments.

In [19]:
from pathlib import Path
import sys
import os
import numpy as np

project_root = Path(os.getcwd()).resolve().parent
sys.path.append(str(project_root))

## 1. Fetch Data
Data is downloaded from [Kaggle Airline Passenger Satisfaction](https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction).
<br>It contains two files: `train.csv` and `test.csv` at a 75%-25% split.
<br>Datasets were downloaded and saved in `data/` of this repo.

In [5]:
import pandas as pd
#Load Data
train_df = pd.read_csv(project_root / "data" / "train.csv")
test_df  = pd.read_csv(project_root / "data" / "test.csv")

In [9]:
print("Train:", train_df.shape, " Test:", test_df.shape)
print("Split Ratio: ", len(test_df)/len(train_df))

Train: (103904, 25)  Test: (25976, 25)
Split Ratio:  0.25


## 2. Data Cleaning

Both files contain the unncessary column `Unnamed: 0` which we will remove.

In [14]:
test_df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,satisfied


In [15]:
train_df.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [10]:
#Remove unncessary column
train_df = train_df.drop(columns=['Unnamed: 0'])
test_df = test_df.drop(columns=['Unnamed: 0'])

We also check for uniqueness of the IDs used in the dataset to ensure no duplications.

In [11]:
print("Train ID uniquness: ", train_df['id'].nunique() == len(train_df))
print("Test ID uniquness: ", test_df['id'].nunique() == len(test_df))

Train ID uniquness:  True
Test ID uniquness:  True


## 3. Combined Data
For demonstration purpose of the full model development pipeline, the train and test data will be combined. This provides flexibility for the developer to customize the paramters of the split.

In [36]:
data = pd.concat([train_df, test_df])
data.to_csv('../data/full_data.csv', index = False)
data.shape

(129880, 24)

In [37]:
print("All ID uniquness: ", data['id'].nunique() == len(data))

All ID uniquness:  True


## 4. Target and Features

The target label of the data is the `satisfaction` column which has values of either: `neutral or dissatisfied` or `satisfied`.
<br>With `satisfied` as the positive label (1) and `neutral or dissatisfied` as the negative label (0), we observe an event rate of ~43%

In [38]:
data['satisfaction'].value_counts()/len(data)

satisfaction
neutral or dissatisfied    0.565537
satisfied                  0.434463
Name: count, dtype: float64

### Data Types

Out of 24 columns, 5 columns are categorical -- `Gender`, `Customer Type`, `Type of Travel`, `Class`, and `satisfaction`. These columns will need one-hot-encoding. 

In [39]:
data.dtypes

id                                     int64
Gender                                object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Inflight wifi service                  int64
Departure/Arrival time convenient      int64
Ease of Online booking                 int64
Gate location                          int64
Food and drink                         int64
Online boarding                        int64
Seat comfort                           int64
Inflight entertainment                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Inflight service                       int64
Cleanliness                            int64
Departure Delay in Minutes             int64
Arrival De

### Data Summary

For the numeric columns, aside from `Age`, `Flight Distance`, `Departure Delay in Minutes` and `Arrival Delay in Minutes`, all of the other variables are survey ratings (1 to 5) based on the passengers' survey responses. All variables have good variablity evidenced by the std per column (none that are close to 0). Hence, no columns will be dropped due to low variance.

In [40]:
data.describe()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129487.000000
mean,64940.500000,39.427957,1190.316392,2.728696,3.057599,2.756876,2.976925,3.204774,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,14.713713,15.091129
std,37493.270818,15.119360,997.452477,1.329340,1.526741,1.401740,1.278520,1.329933,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,38.071126,38.465650
min,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32470.750000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,64940.500000,40.000000,844.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,97410.250000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


### Missing Values

Only `Arrival Delay in Minutes` has missing values at 0.3%. For simplicity, we will impute this with zero being the median value for this column.

In [41]:
data.isna().sum()/len(data)

id                                   0.000000
Gender                               0.000000
Customer Type                        0.000000
Age                                  0.000000
Type of Travel                       0.000000
Class                                0.000000
Flight Distance                      0.000000
Inflight wifi service                0.000000
Departure/Arrival time convenient    0.000000
Ease of Online booking               0.000000
Gate location                        0.000000
Food and drink                       0.000000
Online boarding                      0.000000
Seat comfort                         0.000000
Inflight entertainment               0.000000
On-board service                     0.000000
Leg room service                     0.000000
Baggage handling                     0.000000
Checkin service                      0.000000
Inflight service                     0.000000
Cleanliness                          0.000000
Departure Delay in Minutes        

### Categorical Columns

`Gender` column is balanced between Males and Females. Will be One-Hot-Encoded.

In [42]:
data['Gender'].value_counts()/len(data)

Gender
Female    0.507384
Male      0.492616
Name: count, dtype: float64

82% of customers are tagged as loyal under `Customer Type`. Data dictionary did not disclose the basis for this tagging but could be assumed related to repeat bookings and high volumes/frequencies of transactions. Recommended for One-Hot-Encoding

In [43]:
data['Customer Type'].value_counts()/len(data)

Customer Type
Loyal Customer       0.816908
disloyal Customer    0.183092
Name: count, dtype: float64

`Class` column refers to ticket tiers with varying prices. This can be label encoded with increasing values in the order of `Eco`, `Eco Plus`, and `Business`.

In [44]:
data['Class'].value_counts()/len(data)

Class
Business    0.478596
Eco         0.448945
Eco Plus    0.072459
Name: count, dtype: float64

69% of passengers are business travelers (`Type of Travel`). For One-Hot-Encoding.

In [45]:
data['Type of Travel'].value_counts()/len(data)

Type of Travel
Business travel    0.690584
Personal Travel    0.309416
Name: count, dtype: float64

In [46]:
data = pd.get_dummies(data, columns=['Gender', 'Customer Type', 'Type of Travel', 'satisfaction'], drop_first=True, dtype=int)

### Correlation

In [49]:
corrmat = data.select_dtypes(include=['number']).corr()
corrmat

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,...,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,satisfaction_satisfied
id,1.000000,0.020322,0.095504,-0.023096,-0.002192,0.013400,-0.000113,-0.000510,0.055538,0.052164,...,0.074569,0.079325,0.078793,0.024048,-0.017643,-0.035657,-0.001361,0.001467,-0.000935,0.013182
Age,0.020322,1.000000,0.099459,0.016116,0.036960,0.022565,-0.000398,0.023194,0.207572,0.159136,...,-0.047991,0.033475,-0.051347,0.052565,-0.009041,-0.011248,0.008996,-0.284172,-0.044808,0.134091
Flight Distance,0.095504,0.099459,1.000000,0.006701,-0.018914,0.065165,0.005520,0.057066,0.214825,0.157662,...,0.064855,0.073608,0.059316,0.095648,0.002402,-0.001935,0.003616,-0.226021,-0.266792,0.298085
Inflight wifi service,-0.023096,0.016116,0.006701,1.000000,0.344915,0.714807,0.338573,0.132214,0.457445,0.121513,...,0.120376,0.043762,0.110029,0.131300,-0.015946,-0.017749,0.005901,-0.005757,-0.105865,0.283460
Departure/Arrival time convenient,-0.002192,0.036960,-0.018914,0.344915,1.000000,0.437620,0.447510,0.000687,0.072287,0.008666,...,0.070833,0.091132,0.072195,0.009862,0.000778,-0.000942,0.008995,-0.206873,0.257102,-0.054270
Ease of Online booking,0.013400,0.022565,0.065165,0.714807,0.437620,1.000000,0.460041,0.030514,0.404866,0.028561,...,0.039148,0.008819,0.035373,0.015125,-0.005318,-0.007033,0.005893,-0.018059,-0.134078,0.168877
Gate location,-0.000113,-0.000398,0.005520,0.338573,0.447510,0.460041,1.000000,-0.002872,0.002756,0.002788,...,0.000972,-0.039353,0.000310,-0.005918,0.005973,0.005658,-0.000863,0.004465,-0.029869,-0.002793
Food and drink,-0.000510,0.023194,0.057066,0.132214,0.000687,0.030514,-0.002872,1.000000,0.233500,0.575846,...,0.035321,0.085198,0.035210,0.658054,-0.029164,-0.031715,0.001730,-0.056997,-0.068986,0.211340
Online boarding,0.055538,0.207572,0.214825,0.457445,0.072287,0.404866,0.002756,0.233500,1.000000,0.419253,...,0.083541,0.204238,0.074058,0.329377,-0.019404,-0.022730,-0.045022,-0.189083,-0.224020,0.501749
Seat comfort,0.052164,0.159136,0.157662,0.121513,0.008666,0.028561,0.002788,0.575846,0.419253,1.000000,...,0.074620,0.189979,0.068842,0.679613,-0.027999,-0.030521,-0.030756,-0.156239,-0.127717,0.348829


Most correlated variables to the target are `Online Boarding`, `Type of Travel`, and `Inflight Entertainment`.

In [50]:
corrmat['satisfaction_satisfied']

id                                   0.013182
Age                                  0.134091
Flight Distance                      0.298085
Inflight wifi service                0.283460
Departure/Arrival time convenient   -0.054270
Ease of Online booking               0.168877
Gate location                       -0.002793
Food and drink                       0.211340
Online boarding                      0.501749
Seat comfort                         0.348829
Inflight entertainment               0.398234
On-board service                     0.322205
Leg room service                     0.312424
Baggage handling                     0.248680
Checkin service                      0.237252
Inflight service                     0.244918
Cleanliness                          0.307035
Departure Delay in Minutes          -0.050740
Arrival Delay in Minutes            -0.058275
Gender_Male                          0.011236
Customer Type_disloyal Customer     -0.186017
Type of Travel_Personal Travel    